In [1]:
import os
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import pandas as pd

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "service_acc/alpine-sentry-349911-6e79cefcab88.json"

In [3]:
def predict_tabular_classification_sample(project, endpoint_id,instance_dict,location,api_endpoint):
    
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    
    # for more info on the instance schema, please use get_model_sample.py
    # and look at the yaml found in instance_schema_uri
    instance = json_format.ParseDict(instance_dict, Value())
    instances = [instance]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print(f"Brand + Model Name: {instance_dict['brand']} {instance_dict['model']}")
#     print("deployed_model_id:", response.deployed_model_id)
    
    predictions = response.predictions
    for prediction in predictions:
        result = dict(prediction)
#         print("prediction:", result)
    return response, result

### 

### Get Prediction

In [4]:
df = pd.read_csv('data/structured-data/Price_Prediction.csv')
n = 10
data = df.sample(n)
instances = []
for i in range(n):
    instance = data.iloc[i].to_dict()
    for k,v in instance.items():
        instance[k] = str(v)
    
    instances.append(instance)

In [5]:

for i in range(n):
    response, result = predict_tabular_classification_sample(project="797761434506",
                                                   endpoint_id="7298540592991043584",
                                                   location="us-west2",
                                                   api_endpoint= "us-west2-aiplatform.googleapis.com",
                                                   instance_dict = instances[i])

    for key, val in result.items():
        print(f'{key} price : {round(val)}')
        
    print()
    

Brand + Model Name: Maruti Ciaz
upper_bound price : 1102376
value price : 935951
lower_bound price : 666095

Brand + Model Name: Maruti Alto
value price : 273523
upper_bound price : 314546
lower_bound price : 204371

Brand + Model Name: Maruti Baleno
value price : 681634
upper_bound price : 835419
lower_bound price : 522351

Brand + Model Name: BMW 3
upper_bound price : 2242290
value price : 922491
lower_bound price : 423834

Brand + Model Name: Toyota Innova
lower_bound price : 321132
value price : 627118
upper_bound price : 1166403

Brand + Model Name: Honda City
value price : 577534
upper_bound price : 959971
lower_bound price : 392568

Brand + Model Name: Hyundai Creta
value price : 861030
upper_bound price : 1070504
lower_bound price : 598077

Brand + Model Name: Hyundai Verna
lower_bound price : 249566
value price : 387447
upper_bound price : 841274

Brand + Model Name: Volkswagen Polo
lower_bound price : 288774
value price : 385418
upper_bound price : 581773

Brand + Model Name:

In [6]:
from typing import Dict
from google.cloud import aiplatform_v1beta1

In [7]:
def explain_tabular_sample(project: str,endpoint_id: str,instance_dict: Dict,location: str = "us-central1",
                           api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform_v1beta1.PredictionServiceClient(client_options=client_options)
    
    # The format of each instance should conform to the deployed model's prediction input schema.
    instance = json_format.ParseDict(instance_dict, Value())
    instances = [instance]
    
    # tabular models do not have additional parameters
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.explain(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    explanations = response.explanations
    for explanation in explanations:
        print(" explanation")
        # Feature attributions.
        attributions = explanation.attributions
        for attribution in attributions:
            print("  attribution")
            print("   baseline_output_value:", attribution.baseline_output_value)
            print("   instance_output_value:", attribution.instance_output_value)
            print("   output_display_name:", attribution.output_display_name)
            print("   approximation_error:", attribution.approximation_error)
            print("   output_name:", attribution.output_name)
            output_index = attribution.output_index
            for output_index in output_index:
                print("   output_index:", output_index)
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))

In [8]:
explain_tabular_sample(project="797761434506",
                       endpoint_id="7298540592991043584",
                       location="us-west2",
                       api_endpoint= "us-west2-aiplatform.googleapis.com",
                       instance_dict = instances[0])

response
 deployed_model_id: 3496235870095147008
 explanation
  attribution
   baseline_output_value: 574983.0
   instance_output_value: 935951.5
   output_display_name: 
   approximation_error: 0.0008467534646194617
   output_name: value
   output_index: -1
 prediction: {'value': 935951.4375, 'upper_bound': 1102376.25, 'lower_bound': 666094.9375}
